In [1]:
import mediapipe as mp
import numpy as np
import cv2

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [14]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5,) as pose:
    while cap.isOpened():
        _, frame = cap.read()
        frame = cv2.flip(frame,1)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame.flags.writeable = False
        result = pose.process(frame)
        
        frame.flags.writeable = True
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        
        cv2.imshow("Frame", frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [15]:
mp_drawing.draw_landmarks?

In [4]:
len(result.pose_landmarks.landmark)

33

!['Landmark'](https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png)

In [16]:
result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE.value]

x: 0.28896671533584595
y: 1.808732509613037
z: 0.08631338179111481
visibility: 0.01732664555311203

In [17]:
mp_pose.PoseLandmark.LEFT_ANKLE.value

27

In [18]:
result.pose_landmarks.landmark[12]

x: 0.04632759466767311
y: 0.4792741537094116
z: -0.275138795375824
visibility: 0.9927991628646851

In [19]:
shoulder = [result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x,result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
elbow = [result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].x,result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].y]
wrist = [result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x,result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y] 

In [20]:
print(shoulder, elbow, wrist)

[0.35667547583580017, 0.488629013299942] [0.5113536715507507, 0.6972600221633911] [0.7075040936470032, 0.9151315689086914]


In [21]:
def calculate_angle(a,b,c):
    a = np.array(a) 
    b = np.array(b) 
    c = np.array(c) 
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [22]:
calculate_angle(shoulder, elbow, wrist)

174.55628514473804

In [47]:
cap = cv2.VideoCapture(0)

counter = 0
stage = None
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5,) as pose:
    while cap.isOpened():
        _, frame = cap.read()
        frame = cv2.flip(frame,1)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame.flags.writeable = False
        result = pose.process(frame)
        
        frame.flags.writeable = True
        
        try:
            landmark = result.pose_landmarks.landmark
            shoulder = [landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
            elbow = [landmark[mp_pose.PoseLandmark.LEFT_ELBOW].x, landmark[mp_pose.PoseLandmark.LEFT_ELBOW].y]
            wrist = [landmark[mp_pose.PoseLandmark.LEFT_WRIST].x, landmark[mp_pose.PoseLandmark.LEFT_WRIST].y]
            
            angle = calculate_angle(shoulder, elbow, wrist)
#             print(angle)
            cv2.putText(frame, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            if angle > 150:
                stage = 'Down'
            elif angle < 45 and stage == 'Down':
                counter += 1 
                stage = 'Up'
                
#             print(stage)
            
        except:
            pass
        
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        cv2.rectangle(frame, (0,0), (225,73), (245,117,16), -1)
        cv2.putText(frame, "Score ",(15,12),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (250,250,250), 1, cv2.LINE_AA)
        cv2.putText(frame,str(counter),(10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        cv2.imshow("Frame", frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()